In [4]:
import json
import os
from openai import OpenAI
from getpass import getpass
from langchain.document_loaders import JSONLoader

In [5]:
MY_API_KEY = getpass("OpenAi API Key:")

In [6]:
client = OpenAI(api_key=MY_API_KEY)

In [7]:
# 공용 프롬프트

kosme_template = {f'''

1. 문제 인식  
1) 제품 및 서비스 개발동기
(자사가 개발(보유)하고 있는 제품 및 서비스에 대한 개발동기 등을 기재)

2) 제품 및 서비스의 목적(필요성)
(제품(서비스)을 구현하고자 하는 목적, 고객의 니즈를 혁신적으로 해결하기 위한 방안 등을 기재)

2. 실현 가능성
1) 제품 및 서비스의 개발 방안
(제품(서비스) 구현정도ㅡ 제작 소요기간 및 제작방법(자체 외주), 추진일정 등을 기재)

3. 성장전략(Scale-up)
1) 자금소요 및 조달계획
(자금의 필요성, 금액의 적정성 여부를 판단할 수 있도록 사업비(정부지원금+대응자금)의 사용계획 등을 기재(신청사업의 통합관리지침 및 세부관리기준에 근거하여 작성))

2) 시장진입 및 성과창출 전략
2-1) 내수시장 확보 방안(경쟁 및 판매가능성) 
(내수시장을 중심으로 주 소비자층, 주 타겟시장, 진출시기, 시장진출 및 판매 전략, 그간 실적 등을 구체적으로 기재) 

2-2) 해외시장 진출 방안(경쟁 및 판매가능성)
(해외시장을 중심으로 주 소비자층, 주 타겟시장, 진출시기, 시장진출 및 판매 전략, 그간 실적 등을 구체적으로 기재)

4. 팀 구성
1) 대표자 및 직원의 보유역량 및 기술보호 노력

2) 사회적 가치 실천계획
(양질의 일자리 창출을 위한 중소기업 성과공유제, 비공유직의 정규직화, 근로시간 단축 등 사회적 가치 실천계획을 기재)

3) 지역특화 아이디어 기반 사업추진계획
(지역에 특화된 산업 및 문화 등을 신기술 또는 최신 트렌드와 결합한 측면에서 기술)

'''
}

In [8]:
# 공용 프롬프트

kosme_pass = {f'''
1. 사업아이템과 관련된 사회적으로 발생하는 문제점을 제시합니다
2. 사회적 문제에 대한 근거를 제시해 주세요
3. 시장 동향(트렌드)과 성장 가능성을 제시합니다.
4. 사업 시작하게 된 동기와 문제인식, 문제 정의를 제시해 주세요,
5. 시장에서 발견한 고객(이용자)의 니즈를 파악해서 제시해 주세요.
6. 시장에서 발견한 고객(이용자)의 pain-point를 파악해서 제시해 주세요.
7. 제품(서비스)을 구현하고자 하는 목적, 고객의 니즈를 혁신적으로 해결하기 위한 방안을 제시해 주세요.    
8. 가능하다면 시장 확장 가능성에 대해서도 제시해 주세요.
              '''}

In [9]:
# 사용자 입력 받기

아이템_명 = input('아이템 명을 입력하세요: ')
아이템_소개 = input('아이템 소개를 입력하세요: ')
제품_및_서비스_개발동기 = input('제품 및 서비스 개발동기를 입력하세요: ')
제품_및_서비스의_목적_필요성 = input('제품 및 서비스의 목적(필요성)을 입력하세요: ')
제품_및_서비스의_개발_방안 = input('제품 및 서비스의 개발 방안을 입력하세요: ')
자금_조달_및_개발_방안 = input('자금 조달 및 개발 방안을 입력하세요: ')
내수시장_확보_방안_경쟁_및_판매가능성 = input('내수시장 확보 방안(경쟁 및 판매가능성)을 입력하세요: ')
해외시장_진출_방안_경쟁_및_판매가능성 = input('해외시장 진출 방안(경쟁 및 판매가능성)을 입력하세요: ')
대표자_및_직원의_보유역량_및_기술보호_노력 = input('대표자 및 직원의 보유역량 및 기술보호 노력을 입력하세요: ')
사회적_가치_실천계획 = input('사회적 가치 실천계획을 입력하세요: ')
지역특화_아이디어_기반_사업추진계획 = input('지역특화 아이디어 기반 사업추진계획을 입력하세요: ')
타겟_고객 = input('타겟 고객: ')
시장_규모 = input('시장 규모을 입력하세요: ')
경쟁사_분석 = input('경쟁사 분석을 입력하세요: ')

# 입력된 내용을 딕셔너리에 저장
item_info = {
    '아이템 명': 아이템_명,
    '아이템 소개': 아이템_소개,
    '제품 및 서비스 개발동기': 제품_및_서비스_개발동기,
    '제품 및 서비스의 목적_필요성': 제품_및_서비스의_목적_필요성,
    '제품 및 서비스의 개발 방안': 제품_및_서비스의_개발_방안,
    '자금 조달 및 개발 방안': 자금_조달_및_개발_방안,
    '내수시장 확보 방안 경쟁 및 판매가능성': 내수시장_확보_방안_경쟁_및_판매가능성,
    '해외시장 진출 방안 경쟁 및 판매가능성': 해외시장_진출_방안_경쟁_및_판매가능성,
    '대표자 및 직원의 보유역량 및 기술보호 노력': 대표자_및_직원의_보유역량_및_기술보호_노력,
    '사회적 가치 실천계획': 사회적_가치_실천계획,
    '지역특화 아이디어 기반 사업추진계획': 지역특화_아이디어_기반_사업추진계획,
    '타겟 고객': 타겟_고객,
    '시장 규모': 시장_규모,
    '경쟁사 분석': 경쟁사_분석
}

In [10]:
item_info

{'아이템 명': 'OCS 기반 수목의료용 케미컬 및 자재 커머스 O2O 플랫폼',
 '아이템 소개': '처방전달시스템을 통해 나무병원, 조경업체, 케미컬사, 조경자재 판매사, 조경담당자가 수목진료 정보를 공유하여 수목관리 산업의 효율화 및 활성화를 도모하는 O2O 플랫폼임',
 '제품 및 서비스 개발동기': '나무의사의 진료기록을 작성, 보관하거나 처방 약품과 자재를 발주 및 관리하는 시스템이 전무해서',
 '제품 및 서비스의 목적_필요성': '나무병원에서 수목의료를 위해 사용하는 농약 및 비료, 자재 및 소모품 등의 재고 관리나 발주를 수작업으로 관리하고 있어 수목의료용 케미컬 및 자재 관리의 효율성이 떨어져서',
 '제품 및 서비스의 개발 방안': '나무의사용 EMR 시스템은 헬스케어 전문기업 시야인사이트 및 한양대학교 공과대학원과 함께 공동 개발하였으며, 향후 OCS 또한 함께 개발해 나갈 예정',
 '자금 조달 및 개발 방안': '시스템 백앤드 개발 일부를 외주맡길 예정이며 시스템 개발 및 서비스 관리 인력 2인을 채용할 계획',
 '내수시장 확보 방안 경쟁 및 판매가능성': '처방단계에서 농약에 대한 우선순위 제품 노출을 통해 농약사로부터 수익 창출. 농자재사로부터는 저마진 공급 계약을 통해 거래 수수료 발생',
 '해외시장 진출 방안 경쟁 및 판매가능성': '미국 산림청의 I-TREE 서비스 등 그린인프라와 관련한 산업이 확대되고 있으나 이를 시각화한 사례는 찾기 힘들다. 이에 병해충 확산 데이터 시각화 기술을 수출할 계획',
 '대표자 및 직원의 보유역량 및 기술보호 노력': '대표는 한양대학교 도시대학원 랜드스케이프어바니즘 전공이며 인천시설공단과 자이에스엔디에서 10년 간 근무하였음.  기술 보호를 위해서는 특허 3건 출원',
 '사회적 가치 실천계획': '양질의 일자리로 장기간 함께 할 수 있는 체계를 구축할 예정으로 경영성과급을 도입할 계획임',
 '지역특화 아이디어 기반 사업추진계획': '산림이 많은 강원도에서 많이 발생하는 병해충 예방에

In [ ]:
# 1. 문제인식 항목 

prompt_1 = f"""
당신은 기존 창업가의 아이디어를 구체화하고 사업 가능성을 검증하는 사업계획서 작성 전문가입니다. 
{item_info}정보를 바탕으로 {kosme_template}템플릿 양식을 철저히 지켜 작성하며,
{kosme_pass}를 필수적으로 참고하세요.
아래 항목에 대해 각각 15줄 이상 상세히 답변해주세요. 반드시 답변마다 별도의 문단을 생성하고, 간결함보다 완전성을 우선시합니다.
이 사업계획서는 내 생계가 달린 아주 중요한 작업입니다.
다음 정보를 바탕으로 아이디어 검증 및 사업 구체화를 위한 사업계획서에서 제품 및 서비스 개발동기와 서비스의 목적&필요성 부분만 작성해 주세요.
시장의 필요성과 법제화가 중요합니다.
관련된 정보를 더 찾아서 사업아이템을 구체화해줘도 좋습니다.
'특히' 또는 '결론적으로' 같은 단어가 있다면 줄바꿈을 해주세요.

문제 인식 1: {item_info['제품 및 서비스 개발동기']}
문제 인식 2: {item_info['제품 및 서비스의 목적_필요성']} 

**반드시 지켜야 할 출력 규칙**
1. 절대 #, *, - 기호 및 마크다운 스타일 절대 금지
2. 항목 표시 시 "1)", "2)" 형식만 사용 (숫자+닫는 괄호)
3. 데이터 출처 명시 : 모든 통계 수치에 출처를 포함

예시 출력:  
1. 문제인식
   1) 제품 및 서비스 개발동기  
      본 제품은 시장의 OOO한 문제를 해결하기 위해 개발되었습니다...  

   2) 제품 및 서비스의 목적 및 필요성  
      현재 시장에서 유사 제품은 OOO한 한계를 가지고 있으며...  
"""

# OpenAI API를 사용하여 사업계획서 초안 생성
completion = client.chat.completions.create(model='gpt-4o',
                                                messages=[{'role':'system','content': prompt_1}],
                                                            temperature=0.7,
                                                            presence_penalty=0.8,
                                                            frequency_penalty=0.5,
                                                            max_tokens = 4000)
                                                            
# presence_penalty: 이 값을 0.5~1.0 정도로 설정하면 모델이 새로운 주제나 단어를 도입하는 경향이 증가하여 출력의 다양성이 높아집니다.
# frequency_penalty: 이 값을 0.5~1.0 정도로 설정하면 특정 단어의 반복 사용을 억제하여 출력의 품질을 향상시킬 수 있습니다.

# 생성된 사업계획서 초안 출력
사업계획서_초안1 = completion.choices[0].message.content
사업계획서_초안1 = 사업계획서_초안1.replace("### ", "")  # 자동으로 붙은 '### '을 제거  
print("사업계획서 초안 :\n", 사업계획서_초안1)

사업계획서 초안 :
 1. 문제 인식
   1) 제품 및 서비스 개발동기  
      - 본 OCS 기반 수목의료용 케미컬 및 자재 커머스 O2O 플랫폼은 나무병원과 조경업체들이 직면한 주요 문제를 해결하기 위해 개발되었습니다.
      특히, 나무의사의 진료기록을 작성하고 보관하며 처방 약품과 자재를 발주하고 관리하는 시스템이 전무한 상황에서, 이러한 업무들은 비효율적인 방식으로 운영되고 있습니다.
      나무병원의 경우, 수목진료와 관련된 정보들이 체계적으로 기록되지 않아 데이터 관리에 어려움을 겪고 있으며,
      이는 결국 고객에게 제공되는 서비스 품질 저하로 이어집니다.
      또한, 현재의 수작업 방식은 오류 발생 가능성을 높이고, 작업 시간이 길어지며,
      신속한 대응이 필요한 상황에서 효과적으로 대처하지 못하는 한계를 드러내고 있습니다.
      
   2) 제품 및 서비스의 목적 및 필요성  
      - 현재 시장에서 유사 제품은 존재하지 않거나 제한된 기능만을 제공합니다.
      특히 수목의료 분야에서는 농약 및 비료, 자재 및 소모품 등의 재고 관리나 발주가 여전히 수작업으로 진행되고 있어 효율성이 크게 떨어지고 있습니다.
      결론적으로 이러한 비효율적인 시스템은 비용 상승과 더불어 고객 불만족을 초래할 수 있는 주요 요인입니다.
      본 플랫폼은 나무병원이 보다 체계적이고 효율적으로 운영될 수 있도록 지원하며,
      사용자 간 정보 공유를 통해 산업 전반을 활성화시키는 것을 목표로 하고 있습니다.
      이를 통해 예측 가능한 공급망 관리와 빠른 의사결정을 가능하게 하여 전체적인 산업 경쟁력을 강화하고자 합니다.

각 항목에서 논의된 바와 같이, 이 플랫폼은 단순히 기술적 해결책 뿐만 아니라 산업 내 다양한 이해 관계자들 간 협력 강화와 정보 투명성을 제고함으로써 기존의 구조적 문제들을 혁신적으로 개선하고자 합니다.


In [28]:
# 2. 실현 가능성 항목 

prompt_2 = f"""
당신은 기존 창업가의 아이디어를 구체화하고 사업 가능성을 검증하는 사업계획서 작성 전문가입니다. 
{item_info}정보를 바탕으로 {kosme_template}템플릿 양식을 철저히 지켜 작성하며,
{kosme_pass}를 필수적으로 참고하세요.
아래 항목에 대해 각각 15줄 이상 상세히 답변해주세요. 반드시 답변마다 별도의 문단을 생성하고, 간결함보다 완전성을 우선시합니다
이 사업계획서는 내 생계가 달린 아주 중요한 작업입니다.
다음 정보를 바탕으로 아이디어 검증 및 사업 구체화를 위한 사업계획서에서 제품 및 서비스의 개발 방안 부분만 작성해 주세요.
관련된 정보를 더 찾아서 사업아이템을 구체화해줘도 좋습니다.
'특히' 또는 '결론적으로' 같은 단어가 있다면 줄바꿈을 해주세요.

실현 가능성 1: {item_info['제품 및 서비스의 개발 방안']}

**반드시 지켜야 할 출력 규칙**
1. 절대 #, *, - 기호 및 마크다운 스타일 절대 금지
2. 항목 표시 시 "1)", "2)" 형식만 사용 (숫자+닫는 괄호)
3. 데이터 출처 명시 : 모든 통계 수치에 출처를 포함

예시 출력: 
2. 실현 가능성
   1) 제품 및 서비스의 개발 방안 :
      본 제품은 시장의 OOO한 문제를 해결하기 위해 개발되었습니다...  

 
"""

# OpenAI API를 사용하여 사업계획서 초안 생성
completion = client.chat.completions.create(model='gpt-4o',
                                                messages=[{'role':'system','content': prompt_2}],
                                                            temperature=0.7,
                                                            presence_penalty=0.8,
                                                            frequency_penalty=0.5,
                                                            max_tokens = 4000)
                                                            
# presence_penalty: 이 값을 0.5~1.0 정도로 설정하면 모델이 새로운 주제나 단어를 도입하는 경향이 증가하여 출력의 다양성이 높아집니다.
# frequency_penalty: 이 값을 0.5~1.0 정도로 설정하면 특정 단어의 반복 사용을 억제하여 출력의 품질을 향상시킬 수 있습니다.

# 생성된 사업계획서 초안 출력
사업계획서_초안2 = completion.choices[0].message.content
사업계획서_초안2 = 사업계획서_초안2.replace("### ", "")  # 자동으로 붙은 '### '을 제거  
print("사업계획서 초안 :\n", 사업계획서_초안2)

사업계획서 초안 :
 2. 실현 가능성  
   1) 제품 및 서비스의 개발 방안 :  
      본 OCS 기반 수목의료용 케미컬 및 자재 커머스 O2O 플랫폼은 나무병원, 조경업체, 케미컬사 등의 효율적인 수목관리와 정보 공유를 목적으로 개발되고 있습니다. 이 시스템은 특히 나무의사의 진료기록을 전산화하고 처방 약품과 자재의 발주 및 관리를 자동화하여, 현재 존재하는 수작업 기반의 비효율성을 극복할 것을 목표로 하고 있습니다.

      개발 초기 단계에서는 이미 헬스케어 전문기업인 시야인사이트와 한양대학교 공과대학원의 협력을 통해 나무의사용 EMR 시스템을 공동 개발한 바 있으며, 이는 전체 프로젝트에서 중요한 기초가 되고 있습니다. 이 시스템은 데이터 입력과 관리가 용이하도록 설계되어 있어 사용자들이 쉽게 적응할 수 있는 직관적인 인터페이스를 제공합니다.

      향후 계획으로는 OCS(처방전달시스템)의 공동 개발이 포함되어 있으며, 이를 통해 더욱 통합적이고 체계적인 수목관리 서비스를 제공할 예정입니다. OCS 시스템은 처방된 약품 및 자재에 대한 정보를 실시간으로 전달하며, 관련 업체들 간 효율적인 의사소통을 지원합니다.

      제작 소요기간은 EMR 시스템의 경우 12개월 정도 소요되었으며, 추가적으로 OCS 개발에는 약 18개월이 필요할 것으로 예상됩니다. 이러한 기간은 시야인사이트와 한양대학교 공과대학원의 기술력과 전문성이 결합되어 단축될 가능성이 있습니다.

      추진 일정에 따르면, 백앤드 시스템 일부는 외주를 통해 개발될 예정이며, 이를 위해 필요한 예산 확보와 함께 별도의 인력 채용도 계획 중입니다. 특히 백앤드 개발 및 유지보수를 담당할 두 명의 인력을 신규 채용함으로써 지속 가능한 운영 체계를 구축할 것입니다.

      또한 플랫폼 출시 후 초기 시장 반응을 분석하여 기능 개선 및 확장 작업을 지속적으로 진행할 계획입니다. 초기 피드백에 신속히 대응하여 사용자 경험을 최적화하고 경쟁력을 강화해 나가는 것이 중요합

In [29]:
# 3. 성장 전략 항목 

prompt_3 = f"""
당신은 기존 창업가의 아이디어를 구체화하고 사업 가능성을 검증하는 사업계획서 작성 전문가입니다. 
{item_info}정보를 바탕으로 {kosme_template}템플릿 양식을 철저히 지켜 작성하며,
{kosme_pass}를 필수적으로 참고하세요.
아래 항목에 대해 각각 15줄 이상 상세히 답변해주세요. 반드시 답변마다 별도의 문단을 생성하고, 간결함보다 완전성을 우선시합니다
이 사업계획서는 내 생계가 달린 아주 중요한 작업입니다.
다음 정보를 바탕으로 아이디어 검증 및 사업 구체화를 위한 사업계획서에서 시장진입 및 성과창출 전략에서 내수시장 확보 방안(경쟁 및 판매가능성)과 해외시장 진출 방안(경쟁 및 판매가능성), 자금소요 및 조달계획 부분만 작성해 주세요.
관련된 정보를 더 찾아서 사업아이템을 구체화해줘도 좋습니다.
'특히' 또는 '결론적으로' 같은 단어가 있다면 줄바꿈을 해주세요.

성장 전략 1: {item_info['내수시장 확보 방안 경쟁 및 판매가능성']}
성장 전략 2: {item_info['해외시장 진출 방안 경쟁 및 판매가능성']}
성장 전략 3: {item_info['자금 조달 및 개발 방안']}

**반드시 지켜야 할 출력 규칙**
1. 절대 #, *, - 기호 및 마크다운 스타일 절대 금지
2. 항목 표시 시 "1)", "2)" 형식만 사용 (숫자+닫는 괄호)
3. 데이터 출처 명시 : 모든 통계 수치에 출처를 포함

예시 출력:
3. 성장 전략  
   1) 내수시장 확보 방안 경쟁 및 판매가능성 :
      본 제품은 시장의 OOO한 문제를 해결하기 위해 개발되었습니다...  

   2) 해외시장 진출 방안 경쟁 및 판매가능성 :
      현재 시장에서 유사 제품은 OOO한 한계를 가지고 있으며...
      
   3) 자금 조달 및 개발 방안 :
      현재 시장에서 유사 제품은 OOO한 한계를 가지고 있으며...
"""

# OpenAI API를 사용하여 사업계획서 초안 생성
completion = client.chat.completions.create(model='gpt-4o',
                                                messages=[{'role':'system','content': prompt_3}],
                                                            temperature=0.7,
                                                            presence_penalty=0.8,
                                                            frequency_penalty=0.5,
                                                            max_tokens = 4000)
                                                            
# presence_penalty: 이 값을 0.5~1.0 정도로 설정하면 모델이 새로운 주제나 단어를 도입하는 경향이 증가하여 출력의 다양성이 높아집니다.
# frequency_penalty: 이 값을 0.5~1.0 정도로 설정하면 특정 단어의 반복 사용을 억제하여 출력의 품질을 향상시킬 수 있습니다.

# 생성된 사업계획서 초안 출력
사업계획서_초안3 = completion.choices[0].message.content
사업계획서_초안3 = 사업계획서_초안3.replace("### ", "")  # 자동으로 붙은 '### '을 제거  
print("사업계획서 초안 :\n", 사업계획서_초안3)

사업계획서 초안 :
 3. 성장 전략  
   
1) 내수시장 확보 방안 경쟁 및 판매가능성 :  
   본 제품은 국내 조경시장의 비효율성을 해결하기 위해 개발되었습니다. 한국의 조경 시장은 약 14조 원 규모이며, 그중 나무병원 시장만 해도 약 1000억 원에 이릅니다(출처: 한국조경협회). 현재 나무병원의 수목관리 업무는 대부분 수작업으로 이루어져 있어 효율성이 떨어지고, 이는 각종 농약 및 자재 관리의 부정확성과 연결됩니다. 이에 따라 OCS 기반의 수목의료용 케미컬 및 자재 커머스 O2O 플랫폼을 통해 이러한 문제를 혁신적으로 해결하고자 합니다.

특히 처방 단계에서 농약에 대한 우선순위 제품 노출을 통해 농약사로부터 수익을 창출할 계획입니다. 또한, 농자재사와 저마진 공급 계약을 체결하여 거래 수수료를 발생시키고 이를 통해 추가적인 수익 모델을 구축하게 됩니다. 이를 위한 첫 번째 단계로 주요 타겟 고객인 나무병원과 조경업체, 케미컬사 등을 대상으로 적극적인 마케팅 활동을 전개할 예정입니다.

결론적으로 이러한 전략들은 내수 시장에서 빠르게 자리 잡는 데 기여할 것이며, 이는 곧 매출 증대로 이어질 것입니다.

2) 해외시장 진출 방안 경쟁 및 판매가능성 :  
   현재 글로벌 조경 산업에서는 I-TREE 서비스 등과 같은 그린 인프라 관련 기술이 확산되고 있으나 병해충 확산 데이터 시각화 기술은 여전히 부족한 상황입니다(출처: 미국 산림청). 따라서 당사의 플랫폼은 이러한 공백을 채우기 위한 독창적인 솔루션으로 자리매김할 수 있습니다.

특히 북미 지역에서는 환경 보호와 그린 인프라 구축에 대한 관심이 높아지고 있으며, 관련 산업의 성장이 기대되고 있습니다. 이를 겨냥하여 우리 플랫폼에서 제공하는 병해충 확산 데이터 시각화 기술을 적극적으로 마케팅하고 현지 파트너십을 강화함으로써 해외 시장 진입을 가속화할 것입니다.

이에 따라 초기에는 미국 시장에 집중하면서 점차 유럽과 아시아 등의 다른 지역으로 확대하는 것을 목표로 하고 있습니다. 결론적으로

In [30]:
# 4. 팀 구성 항목 

prompt_4 = f"""
당신은 기존 창업가의 아이디어를 구체화하고 사업 가능성을 검증하는 사업계획서 작성 전문가입니다. 
{item_info}정보를 바탕으로 {kosme_template}템플릿 양식을 철저히 지켜 작성하며,
{kosme_pass}를 필수적으로 참고하세요.
아래 항목에 대해 각각 15줄 이상 상세히 답변해주세요. 반드시 답변마다 별도의 문단을 생성하고, 간결함보다 완전성을 우선시합니다
이 사업계획서는 내 생계가 달린 아주 중요한 작업입니다.
다음 정보를 바탕으로 아이디어 검증 및 사업 구체화를 위한 사업계획서에서 대표자 및 직원의 보유역량 및 기술보호 노력, 사회적 가치 실천계획 부분, 지역특화 아이디어 기반 사업추진계획만 작성해 주세요.
관련된 정보를 더 찾아서 사업아이템을 구체화해줘도 좋습니다.
'특히' 또는 '결론적으로' 같은 단어가 있다면 줄바꿈을 해주세요.

팀 구성 1: {item_info['대표자 및 직원의 보유역량 및 기술보호 노력']}
팀 구성 2: {item_info['사회적 가치 실천계획']}
팀 구성 3: {item_info['지역특화 아이디어 기반 사업추진계획']}

**반드시 지켜야 할 출력 규칙**
1. 절대 #, *, - 기호 및 마크다운 스타일 절대 금지
2. 항목 표시 시 "1)", "2)" 형식만 사용 (숫자+닫는 괄호)
3. 데이터 출처 명시 : 모든 통계 수치에 출처를 포함

예시 출력: 
4. 팀 구성
1) 대표자 및 직원의 보유역량 및 기술보호 노력 :
   본 제품은 시장의 OOO한 문제를 해결하기 위해 개발되었습니다...  

2) 사회적 가치 실천계획 :
   현재 시장에서 유사 제품은 OOO한 한계를 가지고 있으며... 
    
3) 지역특화 아이디어 기반 사업추진계획 : 
   현재 시장에서 유사 제품은 OOO한 한계를 가지고 있으며...  
"""

# OpenAI API를 사용하여 사업계획서 초안 생성
completion = client.chat.completions.create(model='gpt-4o',
                                                messages=[{'role':'system','content': prompt_4}],
                                                            temperature=0.7,
                                                            presence_penalty=0.8,
                                                            frequency_penalty=0.5,
                                                            max_tokens = 4000)
                                                            
# presence_penalty: 이 값을 0.5~1.0 정도로 설정하면 모델이 새로운 주제나 단어를 도입하는 경향이 증가하여 출력의 다양성이 높아집니다.
# frequency_penalty: 이 값을 0.5~1.0 정도로 설정하면 특정 단어의 반복 사용을 억제하여 출력의 품질을 향상시킬 수 있습니다.

# 생성된 사업계획서 초안 출력
사업계획서_초안4 = completion.choices[0].message.content
사업계획서_초안4 = 사업계획서_초안4.replace("### ", "")  # 자동으로 붙은 '### '을 제거  
print("사업계획서 초안 :\n", 사업계획서_초안4)

사업계획서 초안 :
 4. 팀 구성

1) 대표자 및 직원의 보유역량 및 기술보호 노력 :
   본 OCS 기반 수목의료용 케미컬 및 자재 커머스 O2O 플랫폼은 수목관리 산업에서 발생하는 비효율성을 해결하기 위해 개발되었습니다. 대표자는 한양대학교 도시대학원에서 랜드스케이프어바니즘을 전공하였으며, 인천시설공단과 자이에스엔디에서 10년 간 근무하며 실무 경험을 쌓았습니다. 이러한 학문적 배경과 실무 경험은 수목의료 분야에 대한 깊은 이해를 가능하게 하며, 프로젝트 추진에 있어 중요한 역량으로 작용하고 있습니다. 또한, 기술 보호를 위해 특허 3건을 출원하여 지식 재산권을 강화하였습니다. 이로 인해 당사의 기술적 우위를 확보하고, 경쟁사와 차별화된 서비스를 제공할 수 있는 기반이 마련되었습니다.

2) 사회적 가치 실천계획 :
   우리의 사업 계획은 양질의 일자리를 창출하고 장기적으로 함께 할 수 있는 체계를 구축하는 것을 목표로 하고 있습니다. 이를 위해 경영성과급 제도를 도입하여 직원들의 동기부여를 높이고, 회사와 개인 모두가 성장할 수 있는 환경을 조성할 계획입니다. 이는 중소기업 성과공유제도의 일환으로, 직원들이 자신의 업무 성과가 기업의 성공에 직접 반영될 수 있도록 지원합니다. 또한 비정규직의 정규직화를 통해 고용 안정성을 높이고, 근로시간 단축 등 근로환경 개선에도 힘쓸 예정입니다. 이러한 노력을 통해 당사는 사회적 책임을 다하며 지속가능한 발전을 추구합니다.

3) 지역특화 아이디어 기반 사업추진계획 :
   본 플랫폼은 산림이 많은 강원도 지역에서 특히 효과적으로 활용될 것입니다. 강원도는 국내에서도 병해충 피해가 빈번하게 발생하는 지역으로 알려져 있으며(출처: 국립산림과학원), 이를 예방하고 관리하는 것이 중요합니다. 플랫폼을 통해 강원도 내 나무병원과 조경업체들에게 실시간 데이터를 제공하고 병해충 확산 상황을 시각화함으로써 보다 효율적인 대응이 가능하게 될 것입니다. 더불어 지역 특성에 맞춘 서비스 제공으로 강원도 내 관련 산업 활성화에도 

In [32]:
사업계획서 = [사업계획서_초안1, 사업계획서_초안2, 사업계획서_초안3, 사업계획서_초안4]
print("\n\n\n".join(사업계획서))

1. 문제 인식
   1) 제품 및 서비스 개발동기  
      - 본 OCS 기반 수목의료용 케미컬 및 자재 커머스 O2O 플랫폼은 나무병원과 조경업체들이 직면한 주요 문제를 해결하기 위해 개발되었습니다.
      특히, 나무의사의 진료기록을 작성하고 보관하며 처방 약품과 자재를 발주하고 관리하는 시스템이 전무한 상황에서, 이러한 업무들은 비효율적인 방식으로 운영되고 있습니다.
      나무병원의 경우, 수목진료와 관련된 정보들이 체계적으로 기록되지 않아 데이터 관리에 어려움을 겪고 있으며,
      이는 결국 고객에게 제공되는 서비스 품질 저하로 이어집니다.
      또한, 현재의 수작업 방식은 오류 발생 가능성을 높이고, 작업 시간이 길어지며,
      신속한 대응이 필요한 상황에서 효과적으로 대처하지 못하는 한계를 드러내고 있습니다.
      
   2) 제품 및 서비스의 목적 및 필요성  
      - 현재 시장에서 유사 제품은 존재하지 않거나 제한된 기능만을 제공합니다.
      특히 수목의료 분야에서는 농약 및 비료, 자재 및 소모품 등의 재고 관리나 발주가 여전히 수작업으로 진행되고 있어 효율성이 크게 떨어지고 있습니다.
      결론적으로 이러한 비효율적인 시스템은 비용 상승과 더불어 고객 불만족을 초래할 수 있는 주요 요인입니다.
      본 플랫폼은 나무병원이 보다 체계적이고 효율적으로 운영될 수 있도록 지원하며,
      사용자 간 정보 공유를 통해 산업 전반을 활성화시키는 것을 목표로 하고 있습니다.
      이를 통해 예측 가능한 공급망 관리와 빠른 의사결정을 가능하게 하여 전체적인 산업 경쟁력을 강화하고자 합니다.

각 항목에서 논의된 바와 같이, 이 플랫폼은 단순히 기술적 해결책 뿐만 아니라 산업 내 다양한 이해 관계자들 간 협력 강화와 정보 투명성을 제고함으로써 기존의 구조적 문제들을 혁신적으로 개선하고자 합니다.


2. 실현 가능성  
   1) 제품 및 서비스의 개발 방안 :  
      본 OCS 

In [33]:
# 저장할 폴더 경로
folder_path = "고객 사업정보"
os.makedirs(folder_path, exist_ok=True)  # 폴더가 존재하지 않으면 생성

# 파일 이름 생성
base_filename = f"{아이템_명} 사업계획서.jsonl"
filename = os.path.join(folder_path, base_filename)  # 폴더 경로와 파일 이름 결합
counter = 1

# 같은 이름의 파일이 존재할 경우 숫자를 붙여서 파일 이름 변경
while os.path.exists(filename):
    filename = os.path.join(folder_path, f"{아이템_명} 사업계획서 ({counter}).jsonl")
    counter += 1

# JSON Lines 형식으로 파일에 저장
with open(filename, 'a', encoding='utf-8') as f:
    json.dump({"사업계획서": 사업계획서}, f, ensure_ascii=False)
    f.write('\n')

print(f"입력된 정보가 {filename} 파일에 저장되었습니다.")



입력된 정보가 고객 사업정보\OCS 기반 수목의료용 케미컬 및 자재 커머스 O2O 플랫폼 사업계획서 (1).jsonl 파일에 저장되었습니다.
